In [2]:
#Yu Yamaoka
#Function Define
import cv2
import numpy as np
import copy

def obj_detection(mask_path, class_id: int):
    """
        opencvのブロブ検出関数の戻り値が微妙に変わってる気がするのでダウングレード推奨
        最新はcv2.__version == 4.5.1なので pip install -U opencv-python==3.4.13

        N個の物体が入っている一枚の二値画像(height, width, 1)から,
        opencvのブロブ検出関数を利用して (height, widht, N)のNチャネル画像を生成する
        各チャネルが１つの物体のマスク情報となっている

        cls_idxsは各チャネルのマスク情報が示す物体のクラスidのリストとなっている
        ※もとのブログではすべての物体がcell(id=1)だったのでcls_idxsはnp.ones(N)でよかった
    """
    mask = cv2.imread(mask_path, 0)
    _, mask = cv2.threshold(mask, 150, 255, cv2.THRESH_BINARY)

    tmp = cv2.connectedComponentsWithStats(mask)
    data = copy.deepcopy(tmp[1])

    #: 可視化するとブロブ検出関数の意味がよくわかる
    #: plt.imshow(data)
    #: plt.show()

    labels = []
    for label in np.unique(data):
        #: ラベルID==0は背景
        if label == 0:
            continue
        else:
            labels.append(label)

    if len(labels) == 0:
        #: 対象オブジェクトがない場合はNone
        return None, None
    else:
        mask = np.zeros((mask.shape)+(len(labels),), dtype=np.uint8)
        for n, label in enumerate(labels):
            mask[:, :, n] = np.uint8(data == label)
        cls_idxs = np.ones([mask.shape[-1]], dtype=np.int32) * class_id

        return mask, cls_idxs
    